In [ ]:
import pandas as pd
import numpy as np

import os
import pathlib

import cv2 #영상처리에 사용하는 오픈소스 라이브러리, 컴퓨터가 사람 눈처럼 인식할 수 있게 처리
from PIL import Image # 파이썬 이미지 처리 pillow 라이브러리
from tensorflow.keras.preprocessing import image

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator #imagedatagenerater는 이미지를 학습시킬 때 학습 데이터의 양이 적을 경우 학습데이터를 조금씩 변형 시켜서 학습데이터의 양을 늘리는 방식중 하나
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img, load_img
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
from matplotlib import style
style.use('fivethirtyeight')#그래프 스타일 지정

from tqdm.auto import tqdm

#난수 랜덤성 고정
np.random.seed(42)

%matplotlib inline

In [ ]:
cd /content/drive/MyDrive/magnet/GTSRB

/content/drive/.shortcut-targets-by-id/1g8sAN62Ni-dYl7F2MXeUJoyjmxEjibt8/GTSRB


### setup_data

In [ ]:
import numpy as np
import os
import gzip
import urllib.request

from keras.models import load_model

class GTSRB:
    def __init__(self):
        imgs_path = "Train"
        data_list = []
        labels_list = []
        # result_class = [3]

        result_class = [3,7, 9, 10, 11, 12, 13, 17, 18, 25, 35, 38]

        for i in result_class:
            i_path = os.path.join(imgs_path, str(i)) # 3, 7, 9, 10, 11, 12,13, 17, 18, 25, 35, 38
            num = 0
            for img in os.listdir(i_path):
          
                im = Image.open(i_path +'/'+ img)
                im = im.resize((32,32))
                im = np.array(im)
                im = im.reshape((3,32,32)).transpose((1,2,0))
                # im = im.resize((3,32,32))
                # im = np.array(im)

                data_list.append(im/255)
                labels_list.append(i)
                num = num + 1
                if num == 1000:
                    break;


        data = np.array(data_list)
        labels = np.array(labels_list)

        # data = (data.astype(np.float32) - 127.5) / 127.5
        labels = to_categorical(labels)

        VALIDATION_SIZE = 5000
        
        self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(np.array(data), labels, test_size=0.4)    

    @staticmethod
    def print():
        return "GTSRB"


class GTSRBModel:
    def __init__(self, restore, session=None):
        self.num_channels = 3
        self.image_size = 32
        self.num_labels = 12
        
        model = Sequential()

        model.add(Conv2D(64, (3, 3),input_shape=(32, 32, 3)))
        model.add(Activation('relu'))
        model.add(Conv2D(64, (3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        
        model.add(Conv2D(128, (3, 3)))
        model.add(Activation('relu'))
        model.add(Conv2D(128, (3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        
        model.add(Flatten())
        model.add(Dense(256))
        model.add(Activation('relu'))
        model.add(Dense(256))
        model.add(Activation('relu'))
        model.add(Dense(39))

        model.load_weights(restore)

        self.model = model

    def predict(self, data):
        return self.model(data)

In [ ]:
data = GTSRB()

### train_models

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
import os


def train(data, file_name, num_epochs=50, batch_size=128):
    """
    Standard neural network training procedure.
    """
    model = Sequential()

    IMG_HEIGHT = 32
    IMG_WIDTH = 32

    # 첫번째 Convolutional Layer : 입력 데이터로부터 특징을 추출
    model.add(Conv2D(filters=96, kernel_size=3, activation='relu', input_shape=data.x_train.shape[1:]))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(rate=0.25))

    # 두번째 Convolutional Layer
    model.add(Conv2D(filters=192, kernel_size=3, activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(rate=0.25)) # 인풋데이터의 25%를 무작위로 0으로 만듦

    # 세번째 Convolutional Layer
    model.add(Conv2D(filters=192, kernel_size=3, activation='relu')) # 특징을 추출하는 기능을 하는 필터, 비선형 값으로 바꿔주는 activation 함수->relu
    # model.add(GlobalAveragePooling2D())
    model.add(Flatten())

    model.add(Dense(units=64, activation='relu'))
    model.add(Dense(39, activation='softmax'))


    # 모델 컴파일 하기
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()

    # 모델 핏하기
    EPOCHS = 50
    model.fit(data.x_train, data.y_train,
              validation_data = (data.x_test, data.y_test), 
              epochs=EPOCHS, steps_per_epoch=60
              )

    if file_name != None:
        model.save(file_name)

    return model


if not os.path.isdir('models'):
    os.makedirs('models')

model = train(data, "models/example_classifier2", num_epochs=30)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 96)        2688      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 96)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 15, 15, 96)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 192)       166080    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 192)        0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 6, 6, 192)         0

60/60 [==============================] - 0s 5ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.0463 - val_accuracy: 0.8324


In [ ]:
loss, accuracy = classifier.model.evaluate(data.x_test, data.y_test)

print('test set accuracy: ', accuracy * 100)

12/12 [==============================] - 0s 6ms/step - loss: 0.0463 - accuracy: 0.8324
test set accuracy:  83.24176073074341


### defensvie_model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
import os
import numpy as np
from keras.layers.core import Lambda
from keras.layers.merge import Average, add
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, AveragePooling2D
from keras.models import Model
import keras.regularizers as regs


class DenoisingAutoEncoder:
    def __init__(self, image_shape,
                 structure,
                 v_noise=0.0,
                 activation="relu",
                 model_dir="./defensive_models/",
                 reg_strength=0.0):
        """
        Denoising autoencoder.

        image_shape: Shape of input image. e.g. 28, 28, 1.
        structure: Structure of autoencoder.
        v_noise: Volume of noise while training.
        activation: What activation function to use.
        model_dir: Where to save / load model from.
        reg_strength: Strength of L2 regularization.
        """
        h, w, c = image_shape
        self.image_shape = image_shape # shape of input image (32,32,3)
        self.model_dir = model_dir 
        self.v_noise = v_noise

        input_img = Input(shape=self.image_shape)
        x = input_img

        
        for layer in structure: 
            if isinstance(layer, int):
                x = Conv2D(layer, (3, 3), activation=activation, padding="same",
                           activity_regularizer=regs.l2(reg_strength))(x)
            elif layer == "max":
                x = MaxPooling2D((2, 2), padding="same")(x)
            elif layer == "average":
                x = AveragePooling2D((2, 2), padding="same")(x)
            else:
                print(layer, "is not recognized!")
                exit(0)

        for layer in reversed(structure):
            if isinstance(layer, int):
                x = Conv2D(layer, (3, 3), activation=activation, padding="same",
                           activity_regularizer=regs.l2(reg_strength))(x)
            elif layer == "max" or layer == "average":
                x = UpSampling2D((2, 2))(x)

        decoded = Conv2D(c, (3, 3), activation='sigmoid', padding='same',
                         activity_regularizer=regs.l2(reg_strength))(x)
        self.model = Model(input_img, decoded)

    def train(self, data, archive_name, num_epochs=100, batch_size=32):
        self.model.compile(loss='mean_squared_error',
                           metrics=['mean_squared_error'],
                           optimizer='adam')
        print(data.x_train.shape)
        noise = self.v_noise * np.random.normal(size=np.shape(data.x_train))
        noisy_train_data = data.x_train + noise
        noisy_train_data = np.clip(noisy_train_data, 0.0, 1.0)

        self.model.fit(noisy_train_data, data.x_train,
                       batch_size=batch_size,
                       validation_data=(data.x_test, data.x_test),
                       epochs=num_epochs,
                       shuffle=True)

        print(os.path.join(self.model_dir, archive_name))
        self.model.save(os.path.join(self.model_dir, archive_name))

    def load(self, archive_name, model_dir=None):
        if model_dir is None: model_dir = self.model_dir
        self.model.load_weights(os.path.join(model_dir, archive_name))


class PackedAutoEncoder:
    def __init__(self, image_shape, structure, data,
                 v_noise=0.1, n_pack=2, pre_epochs=3, activation="relu",
                 model_dir="./defensive_models/"):
        """
        Train different autoencoders.
        Demo code for graybox scenario.

        pre_epochs: How many epochs do we train before fine-tuning.
        n_pack: Number of autoencoders we want to train at once.
        """
        self.v_noise = v_noise
        self.n_pack = n_pack
        self.model_dir = model_dir
        pack = []



        for i in range(n_pack):
            dae = DenoisingAutoEncoder(image_shape, structure, v_noise=v_noise,
                                       activation=activation, model_dir=model_dir)
            dae.train(data, "", num_epochs=pre_epochs)
            pack.append(dae.model)


        shared_input = Input(shape=image_shape, name="shared_input")
        outputs = [dae(shared_input) for dae in pack]
        avg_output = Average()(outputs)
        delta_outputs = [add([avg_output, Lambda(lambda x: -x)(output)])
                         for output in outputs]

        self.model = Model(inputs=shared_input, outputs=outputs+delta_outputs)

    def train(self, data, archive_name, alpha, num_epochs=10, batch_size=32):
        noise = self.v_noise * np.random.normal(size=np.shape(data.x_train))
        noisy_train_data = data.x_train + noise
        noisy_train_data = np.clip(noisy_train_data, 0.0, 1.0)

        train_zeros = [np.zeros_like(data.x_train)] * self.n_pack
        val_zeros = [np.zeros_like(data.x_test)] * self.n_pack

        self.model.compile(loss="mean_squared_error", optimizer="adam",
                           loss_weights=[1.0]*self.n_pack + [-alpha]*self.n_pack)

        self.model.fit(noisy_train_data,
                       [data.x_train]*self.n_pack + train_zeros,
                       batch_size=batch_size,
                       validation_data=(data.x_test,
                            [data.x_test]*self.n_pack+val_zeros),
                       epochs=num_epochs,
                       shuffle=True)

        for i in range(self.n_pack):
            model = Model(self.model.input, self.model.outputs[i])
            self.model.save("")
            print(os.path.join(self.model_dir, archive_name+"_"+str(i)))
            self.model.save(os.path.join(self.model_dir, archive_name+"_"+str(i)))

    def load(self, archive_name, model_dir=None):
        if model_dir is None: model_dir = self.model_dir
        self.model.load_weights(os.path.join(model_dir, archive_name))

In [ ]:
DAE = DenoisingAutoEncoder
PAE = PackedAutoEncoder

shape = [32, 32, 3]
combination_I = [3, "average", 3]
combination_II = [3]
activation = "sigmoid"
reg_strength = 1e-9
epochs = 500

# data = GTSRB()

# AE_II = PAE(shape, combination_II, data, v_noise=0.025, activation=activation, n_pack=8)
# AE_II.train(data, "0825_8_PAE_GTSRB_II", alpha=.2, num_epochs=epochs)

# AE_II = PAE(shape, combination_II, data, v_noise=0.025, activation=activation)
# AE_II.train(data, "0826_PAE_GTSRB_II", alpha=.2, num_epochs=epochs)    

# AE_II = PAE(shape, combination_II, data, v_noise=0.025, activation=activation, n_pack=32)
# AE_II.train(data, "O32_PAE_GTSRB_II", alpha=.2, num_epochs=epochs)  

AE_I = DAE(shape, combination_I, v_noise=0.1, activation=activation,
           reg_strength=reg_strength)
AE_I.train(data, "500_DAE_GTSRB_I", num_epochs=epochs)

AE_II = DAE(shape, combination_II, v_noise=0.1, activation=activation,
            reg_strength=reg_strength)
AE_II.train(data, "500_DAE_GTSRB_II", num_epochs=epochs)

(7023, 32, 32, 3)
Epoch 1/500
220/220 [==============================] - 11s 10ms/step - loss: 0.0867 - mean_squared_error: 0.0866 - val_loss: 0.0756 - val_mean_squared_error: 0.0756
Epoch 2/500
220/220 [==============================] - 2s 8ms/step - loss: 0.0730 - mean_squared_error: 0.0730 - val_loss: 0.0662 - val_mean_squared_error: 0.0662
Epoch 3/500
220/220 [==============================] - 2s 8ms/step - loss: 0.0518 - mean_squared_error: 0.0518 - val_loss: 0.0467 - val_mean_squared_error: 0.0467
Epoch 4/500
220/220 [==============================] - 2s 8ms/step - loss: 0.0448 - mean_squared_error: 0.0448 - val_loss: 0.0443 - val_mean_squared_error: 0.0443
Epoch 5/500
220/220 [==============================] - 2s 8ms/step - loss: 0.0431 - mean_squared_error: 0.0431 - val_loss: 0.0428 - val_mean_squared_error: 0.0428
Epoch 6/500
220/220 [==============================] - 2s 8ms/step - loss: 0.0417 - mean_squared_error: 0.0417 - val_loss: 0.0414 - val_mean_squared_error: 0.0414
Ep

In [ ]:
loss, accuracy = classfier.model.evaluate(data.x_test, data.y_test)

print('test set accuracy: ', accuracy * 100)

In [ ]:
## utils.py -- utility functions
##
## Copyright (C) 2017, Dongyu Meng <zbshfmmm@gmail.com>.
##
## This program is licenced under the BSD 2-Clause licence,
## contained in the LICENCE file in this directory.

import pickle
import os
import numpy as np


def prepare_data(dataset, idx):
    """
    Extract data from index.

    dataset: Full, working dataset. Such as MNIST().
    idx: Index of test examples that we care about.
    return: X, targets, Y
    """
    return dataset.x_test[idx], dataset.y_test[idx], np.argmax(dataset.y_test[idx], axis=1)


def save_obj(obj, name, directory='./attack_data/'):
    with open(os.path.join(directory, name + '.pkl'), 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


def load_obj(name, directory='./attack_data/'):
    if name.endswith(".pkl"): name = name[:-4]
    with open(os.path.join(directory, name + '.pkl'), 'rb') as f:
        return pickle.load(f)

In [ ]:
## setup_mnist.py -- mnist data and model loading code
##
## Copyright (C) 2016, Nicholas Carlini <nicholas@carlini.com>.
##
## This program is licenced under the BSD 2-Clause licence,
## contained in the LICENCE file in this directory.

## Modified for MagNet's use.

## worker.py -- evaluation code
##
## Copyright (C) 2017, Dongyu Meng <zbshfmmm@gmail.com>.
##
## This program is licenced under the BSD 2-Clause licence,
## contained in the LICENCE file in this directory.

import matplotlib
matplotlib.use('Agg')
from scipy.stats import entropy
from numpy.linalg import norm
from matplotlib.ticker import FuncFormatter
from keras.models import Sequential, load_model
from keras.activations import softmax
from keras.layers import Lambda
import numpy as np
import pylab
import os
import matplotlib.pyplot as plt


class AEDetector:
    def __init__(self, path, p=1):
        """
        Error based detector.
        Marks examples for filtering decisions.

        path: Path to the autoencoder used.
        p: Distance measure to use.
        """
        self.model = load_model(path)
        self.path = path
        self.p = p

    def mark(self, X):
        diff = np.abs(X - self.model.predict(X))
        marks = np.mean(np.power(diff, self.p), axis=(1,2,3))
        return marks

    def print(self):
        return "AEDetector:" + self.path.split("/")[-1]


class IdReformer:
    def __init__(self, path="IdentityFunction"):
        """
        Identity reformer.
        Reforms an example to itself.
        """
        self.path = path
        self.heal = lambda X: X

    def print(self):
        return "IdReformer:" + self.path


class SimpleReformer:
    def __init__(self, path):
        """
        Reformer.
        Reforms examples with autoencoder. Action of reforming is called heal.

        path: Path to the autoencoder used.
        """
        self.model = load_model(path)
        self.path = path

    def heal(self, X):
        X = self.model.predict(X)
        return np.clip(X, 0.0, 1.0)

    def print(self):
        return "SimpleReformer:" + self.path.split("/")[-1]


def JSD(P, Q):
    _P = P / norm(P, ord=1)
    _Q = Q / norm(Q, ord=1)
    _M = 0.5 * (_P + _Q)
    return 0.5 * (entropy(_P, _M) + entropy(_Q, _M))


class DBDetector:
    def __init__(self, reconstructor, prober, classifier, option="jsd", T=1):
        """
        Divergence-Based Detector.

        reconstructor: One autoencoder.
        prober: Another autoencoder.
        classifier: Classifier object.
        option: Measure of distance, jsd as default.
        T: Temperature to soften the classification decision.
        """
        self.prober = prober
        self.reconstructor = reconstructor
        self.classifier = classifier
        self.option = option
        self.T = T

    def mark(self, X):
        return self.mark_jsd(X)

    def mark_jsd(self, X):
        Xp = self.prober.heal(X)
        Xr = self.reconstructor.heal(X)
        Pp = self.classifier.classify(Xp, option="prob", T=self.T)
        Pr = self.classifier.classify(Xr, option="prob", T=self.T)

        marks = [(JSD(Pp[i], Pr[i])) for i in range(len(Pr))]
        return np.array(marks)

    def print(self):
        return "Divergence-Based Detector"


class Classifier:
    def __init__(self, classifier_path):
        """
        Keras classifier wrapper.
        Note that the wrapped classifier should spit logits as output.

        classifier_path: Path to Keras classifier file.
        """
        self.path = classifier_path
        self.model = load_model(classifier_path)
        self.softmax = Sequential()
        self.softmax.add(Lambda(lambda X: softmax(X, axis=1), input_shape=(10,)))

    def classify(self, X, option="logit", T=1):
        if option == "logit":
            return self.model.predict(X)
        if option == "prob":
            logits = self.model.predict(X)/T
            return self.softmax.predict(logits)

    def print(self):
        return "Classifier:"+self.path.split("/")[-1]


class Operator:
    def __init__(self, data, classifier, det_dict, reformer):
        """
        Operator.
        Describes the classification problem and defense.

        data: Standard problem dataset. Including train, test, and validation.
        classifier: Target classifier.
        reformer: Reformer of defense.
        det_dict: Detector(s) of defense.
        """
        self.data = data
        self.classifier = classifier
        self.det_dict = det_dict
        self.reformer = reformer
        self.normal = self.operate(AttackData(data.x_train, np.argmax(data.y_train, axis=1), "Normal"))
        

    def get_thrs(self, drop_rate):
        """
        Get filtering threshold by marking validation set.
        """
        thrs = dict()
        for name, detector in self.det_dict.items():
            num = int(len(data.x_test) * drop_rate[name])
            marks = detector.mark(data.x_test)
            marks = np.sort(marks)
            thrs[name] = marks[-num]
        return thrs

    def operate(self, untrusted_obj):
        """
        For untrusted input(normal or adversarial), classify original input and
        reformed input. Classifier is unaware of the source of input.

        untrusted_obj: Input data.
        """
        X = untrusted_obj.data
        Y_true = untrusted_obj.labels

        X_prime = self.reformer.heal(X)
        Y = np.argmax(self.classifier.classify(X), axis=1)
        Y_judgement = (Y == Y_true[:len(X_prime)])
        Y_prime = np.argmax(self.classifier.classify(X_prime), axis=1)


        Y_prime_judgement = (Y_prime == Y_true[:len(X_prime)])

        return np.array(list(zip(Y_judgement, Y_prime_judgement)))

    def filter(self, X, thrs):
        """
        untrusted_obj: Untrusted input to test against.
        thrs: Thresholds.

        return:
        all_pass: Index of examples that passed all detectors.
        collector: Number of examples that escaped each detector.
        """
        collector = dict()
        all_pass = np.array(range(10000))
        for name, detector in self.det_dict.items():
            marks = detector.mark(X)
            idx_pass = np.argwhere(marks < thrs[name])
            collector[name] = len(idx_pass)
            all_pass = np.intersect1d(all_pass, idx_pass)
        return all_pass, collector

    def print(self):
        components = [self.reformer, self.classifier]
        return " ".join(map(lambda obj: getattr(obj, "print")(), components))


class AttackData:
    def __init__(self, examples, labels, name=""):
        """
        Input data wrapper. May be normal or adversarial.

        examples: Path or object of input examples.
        labels: Ground truth labels.
        """
        # if isinstance(examples, str): 
        #   self.data = load_obj(examples)
        # else: 
        self.data = examples
        self.labels = labels
        self.name = name

    def print(self):
        return "Attack:"+self.name


class Evaluator:
    def __init__(self, operator, untrusted_data, graph_dir="./graph"):
        """
        Evaluator.
        For strategy described by operator, conducts tests on untrusted input.
        Mainly stats and plotting code. Most methods omitted for clarity.

        operator: Operator object.
        untrusted_data: Input data to test against.
        graph_dir: Where to spit the graphs.
        """
        self.operator = operator
        self.untrusted_data = untrusted_data
        self.graph_dir = graph_dir
        self.data_package = operator.operate(untrusted_data)

    def bind_operator(self, operator):
        self.operator = operator
        self.data_package = operator.operate(self.untrusted_data)

    def load_data(self, data):
        self.untrusted_data = data
        self.data_package = self.operator.operate(self.untrusted_data)

    def get_normal_acc(self, normal_all_pass):
        """
        Break down of who does what in defense. Accuracy of defense on normal
        input.

        both: Both detectors and reformer take effect
        det_only: detector(s) take effect
        ref_only: Only reformer takes effect
        none: Attack effect with no defense
        """
        normal_tups = self.operator.normal
        num_normal = len(normal_tups)
        filtered_normal_tups = normal_tups[normal_all_pass]

        both_acc = sum(1 for _, XpC in filtered_normal_tups if XpC)/num_normal
        det_only_acc = sum(1 for XC, XpC in filtered_normal_tups if XC)/num_normal
        ref_only_acc = sum([1 for _, XpC in normal_tups if XpC])/num_normal
        none_acc = sum([1 for XC, _ in normal_tups if XC])/num_normal

        return both_acc, det_only_acc, ref_only_acc, none_acc

    def get_attack_acc(self, attack_pass):
        attack_tups = self.data_package
        num_untrusted = len(attack_tups)
        filtered_attack_tups = attack_tups[attack_pass]

        both_acc = 1 - sum(1 for _, XpC in filtered_attack_tups if not XpC)/num_untrusted
        det_only_acc = 1 - sum(1 for XC, XpC in filtered_attack_tups if not XC)/num_untrusted
        ref_only_acc = sum([1 for _, XpC in attack_tups if XpC])/num_untrusted
        none_acc = sum([1 for XC, _ in attack_tups if XC])/num_untrusted
        return both_acc, det_only_acc, ref_only_acc, none_acc

    def plot_various_confidences(self, graph_name, drop_rate,
                                 idx_file="example_idx",
                                 confs=(0.0, 10.0, 20.0, 30.0, 40.0),
                                 get_attack_data_name=lambda c: "example_carlini_"+str(c)):
        """
        Test defense performance against Carlini L2 attack of various confidences.

        graph_name: Name of graph file.
        drop_rate: How many normal examples should each detector drops?
        idx_file: Index of adversarial examples in standard test set.
        confs: A series of confidence to test against.
        get_attack_data_name: Function mapping confidence to corresponding file.
        """
        pylab.rcParams['figure.figsize'] = 6, 4
        fig = plt.figure(1, (6, 4))
        ax = fig.add_subplot(1, 1, 1)

        idx = original_labels_list
        X, _, Y = prepare_data(self.operator.data, idx)


        det_only = []
        ref_only = []
        both = []
        none = []

        print("\n==========================================================")
        print("Drop Rate:", drop_rate)
        thrs = self.operator.get_thrs(drop_rate)

        all_pass, _ = self.operator.filter(self.operator.data.x_train, thrs)
        all_on_acc, _, _, _ = self.get_normal_acc(all_pass)

        print("Classification accuracy with all defense on:", all_on_acc)

        for confidence in confs:
            f = get_attack_data_name(confidence)
            # self.load_data(AttackData(f, Y, "Carlini L2 " + str(confidence)))

            print("----------------------------------------------------------")
            print("Confidence:", confidence)
            all_pass, detector_breakdown = self.operator.filter(self.untrusted_data.data, thrs)
            both_acc, det_only_acc, ref_only_acc, none_acc = self.get_attack_acc(all_pass)
            print(detector_breakdown)
            both.append(both_acc)
            det_only.append(det_only_acc)
            ref_only.append(ref_only_acc)
            none.append(none_acc)

        size = 2.5

        print("With detector & reformer: ", both_acc)
        print("With detector: ",det_only_acc)
        print("With reformer: ", ref_only_acc)
        print("No Defense: ",none_acc)

        plt.plot(confs, none, c="green", label="No Defense", marker="x", markersize=size,alpha=0.5)
        plt.plot(confs, det_only, c="orange", label="With detector", marker="o", markersize=size,alpha=0.5)
        plt.plot(confs, ref_only, c="blue", label="With reformer", marker="^", markersize=size,alpha=0.5)
        plt.plot(confs, both, c="red", label="With detector & reformer", marker="s", markersize=size,alpha=0.5)

        pylab.legend(loc='lower left', bbox_to_anchor=(0.02, 0.1), prop={'size':8})
        plt.grid(linestyle='dotted')
        plt.xlabel(r"Confidence in Carlini $L^2$ attack")
        plt.ylabel("Classification accuracy")
        plt.xlim(min(confs)-1.0, max(confs)+1.0)
        plt.ylim(-0.05, 1.05)
        ax.yaxis.set_major_formatter(FuncFormatter('{0:.0%}'.format))

        save_path = os.path.join(self.graph_dir, graph_name+".pdf")
        plt.savefig(save_path)
        plt.clf()

    def print(self):
        return " ".join([self.operator.print(), self.untrusted_data.print()])

In [ ]:
imgs_path = "attacked0.02_GTSRB/attacked/"
test_info = pd.read_csv("attacked0.02_GTSRB/predictions_test.csv")

data_list = []
original_labels_list = []
attacked_labels_list = []
label_dict = {0:3, 1:7, 2:9, 3:10, 4:11, 5:12, 6:13, 7:17, 8:18, 9:25, 10:35, 11:38} # mapping (fsgm 공격 이미지 라벨 0,1.2......) -> 실제 이미지 라벨 (3,7,9.....)

num = 0
for img in tqdm(os.listdir(imgs_path)[:1000]):
  im = Image.open(imgs_path+"/"+img)

  im = im.resize((32,32))
  im = np.array(im)
  im = im.reshape((3,32,32))
  im = im.transpose((1,2,0))
  # im = im.resize((32,32))
  # im = np.array(im)


  original_labels_list.append(label_dict[int(test_info[test_info['Unnamed: 0']==int(img[:-4])]['Original'])]) # 기존 label
  attacked_labels_list.append(label_dict[int(test_info[test_info['Unnamed: 0']==int(img[:-4])]['Attacked'])]) # 공격 label
  data_list.append(im/255)
        
attacked_data = np.array(data_list)
# attacked_data = (attacked_data.astype(np.float32) - 127.5) / 127.5

original_labels_list = np.array(original_labels_list)
attacked_labels_list = np.array(attacked_labels_list)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
original_labels_list2 = to_categorical(original_labels_list)
attacked_labels_list2 = to_categorical(attacked_labels_list)

In [ ]:
import tensorflow as tf
import keras

keras.backend.set_learning_phase(False)
sess = keras.backend.get_session()

/usr/local/lib/python3.7/dist-packages/keras/backend.py:450: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [ ]:
classifier = Classifier("./models/example_classifier2")
loss, accuracy = classifier.model.evaluate(attacked_data, original_labels_list2)

32/32 [==============================] - 0s 4ms/step - loss: 0.1733 - accuracy: 0.6340


In [ ]:
loss, accuracy = classifier.model.evaluate(attacked_data, attacked_labels_list2)

32/32 [==============================] - 0s 6ms/step - loss: 0.2038 - accuracy: 0.5100


## 1

In [ ]:
DAE_detector_I = AEDetector("./defensive_models/0826_DAE_GTSRB_I", p=2)
DAE_detector_II = AEDetector("./defensive_models/0826_DAE_GTSRB_II", p=1)
DAE_reformer = SimpleReformer("./defensive_models/0826_DAE_GTSRB_II")


DAE_id_reformer = IdReformer()
DAE_classifier = Classifier("./models/example_classifier2")

DAE_detector_dict = dict()
DAE_detector_dict["I"] = DAE_detector_I
DAE_detector_dict["II"] = DAE_detector_II

# data = GTSRB()

dr = dict([("II"+str(i),.005) for i in range(100)]+[("JSD"+str(i),.01) for i in range(100)])

DAE_operator = Operator(data, DAE_classifier, DAE_detector_dict, DAE_reformer)
DAE_testAttack = AttackData(attacked_data, original_labels_list, "GTSRB FSGM")

DAE_evaluator = Evaluator(DAE_operator, DAE_testAttack)
DAE_evaluator.plot_various_confidences("defense_performance", drop_rate={"I": 0.001, "II": 0.001})


Drop Rate: {'I': 0.001, 'II': 0.001}
Classification accuracy with all defense on: 0.0
----------------------------------------------------------
Confidence: 0.0
{'I': 0, 'II': 0}
----------------------------------------------------------
Confidence: 10.0
{'I': 0, 'II': 0}
----------------------------------------------------------
Confidence: 20.0
{'I': 0, 'II': 0}
----------------------------------------------------------
Confidence: 30.0
{'I': 0, 'II': 0}
----------------------------------------------------------
Confidence: 40.0
{'I': 0, 'II': 0}


In [ ]:
DAE_evaluator = Evaluator(DAE_operator, DAE_testAttack)
DAE_evaluator.plot_various_confidences("defense_performance", drop_rate={"I": 0.001, "II": 0.001})


Drop Rate: {'I': 0.001, 'II': 0.001}
Classification accuracy with all defense on: 0.0
----------------------------------------------------------
Confidence: 0.0
{'I': 0, 'II': 0}
----------------------------------------------------------
Confidence: 10.0
{'I': 0, 'II': 0}
----------------------------------------------------------
Confidence: 20.0
{'I': 0, 'II': 0}
----------------------------------------------------------
Confidence: 30.0
{'I': 0, 'II': 0}
----------------------------------------------------------
Confidence: 40.0
{'I': 0, 'II': 0}
With detector & reformer:  1.0
With detector:  1.0
With reformer:  0.595
No Defense:  0.634


In [ ]:
print('with reformer')
predict = DAE_reformer.model.predict(attacked_data)
predicted = np.argmax(classifier.model.predict(predict),axis=1)
print(np.mean(predicted==original_labels_list))#,predicted)

with reformer
0.595


In [ ]:
print('without reformer')
predicted = np.argmax(classifier.model.predict(attacked_data),axis=1)
print(np.mean(predicted==original_labels_list))#,predicted)

without reformer
0.634


## 2

In [ ]:
attacking = False

# prefix = "O_" 
# if attacking else ""

prefix = "0826_"

detector_I = []
detector_II = [AEDetector("./defensive_models/"+prefix+"PAE_GTSRB_II_"+str(i), p=1) for i in range(2)]
reformer = [SimpleReformer("./defensive_models/"+prefix+"PAE_GTSRB_II_"+str(i)) for i in range(2)]


# detector_I =[]
# detector_II = AEDetector("./defensive_models/"+prefix+"PAE_GTSRB_II_0", p=1)
# reformer = SimpleReformer("./defensive_models/"+prefix+"PAE_GTSRB_II_0")

id_reformer = IdReformer()
classifier = Classifier("./models/example_classifier2")

detector_JSD = [DBDetector(id_reformer, ref, classifier, T=10) for i,ref in enumerate(reformer)]
detector_JSD += [DBDetector(id_reformer, ref, classifier, T=40) for i,ref in enumerate(reformer)]


detector_dict = dict()
for i,det in enumerate(detector_I):
    detector_dict["I"+str(i)] = det
for i,det in enumerate(detector_II):
    detector_dict["II"+str(i)] = det
for i,det in enumerate(detector_JSD):
    detector_dict["JSD"+str(i)] = det

# data = GTSRB()

dr = dict([("II"+str(i),.005) for i in range(100)]+[("JSD"+str(i),.01) for i in range(100)])

detector_JSD = [DBDetector(id_reformer, ref, classifier, T=10) for i,ref in enumerate(reformer)]
detector_JSD += [DBDetector(id_reformer, ref, classifier, T=40) for i,ref in enumerate(reformer)]

In [ ]:
operator = Operator(data, classifier, detector_dict, reformer[0])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:194: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


ValueError: ignored

In [ ]:
testAttack = AttackData(attacked_data, original_labels_list, "GTSRB FSGM")

In [ ]:
evaluator = Evaluator(operator, testAttack)
evaluator.plot_various_confidences("defense_performance", drop_rate={"I": 0.001, "II": 0.001})

In [ ]:
print('with reformer')
predict = reformer[0].model.predict(attacked_data)
predicted = np.argmax(classifier.model.predict(predict),axis=1)
print(np.mean(predicted==original_labels_list))#,predicted)

with reformer


ValueError: ignored